In [1]:
import keras.backend.tensorflow_backend as K
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import nltk
from common_helper import *
from nltk_helper import *
from keras_helper import *
from vectorizer_helper import *

/root/anaconda3/envs/ml_python/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dr = "../input/train.json"
test_dr = "../input/test.json"
traindf, testdf = data_load(train_dr, test_dr)

In [3]:
_x_train_word, y_train = word_data_preprocessing(traindf)
x_test_word, _ = word_data_preprocessing(testdf)

In [4]:
_x_train_token, y_train = token_data_preprocessing(traindf)
x_test_token, _ = token_data_preprocessing(testdf)

In [5]:
_x_train_char, y_train = char_data_preprocessing(traindf)
x_test_char, _ = char_data_preprocessing(testdf)

In [6]:
x_train_word, x_test_word, word_tfidf = tfidf_vectorizer(_x_train_word, x_test_word, ",")
x_train_token, x_test_token, token_tfidf = tfidf_vectorizer(_x_train_token, x_test_token, ",")
x_train_char, x_test_char, char_cntvec = char_vectorizer(_x_train_char, x_test_char, "char")

In [7]:
#column = ['spanish','irish','russian','chinese','brazilian','vietnamese','greek','british','filipino', 'korean']
column = ['korean']

In [8]:
# x_train_word, x_test_word = jump_value(traindf, column, word_tfidf, _x_train_word, x_train_word, x_test_word, 1.5, 300000, 100000)
# x_train_token, x_test_token = jump_value(traindf, column, token_tfidf, _x_train_token, x_train_token, x_test_token, 1.5, 300000, 100000)

In [9]:
lb = preprocessing.LabelBinarizer()
lb.fit(traindf.cuisine.values)
y_train = lb.transform(traindf.cuisine.values)

In [10]:
x_train = np.concatenate((x_train_word, x_train_token), axis=1)
x_test = np.concatenate((x_test_word, x_test_token), axis=1)
x_train = np.concatenate((x_train, x_train_char), axis=1)
x_test = np.concatenate((x_test, x_test_char), axis=1)

In [11]:
ensemble_train = []
ensemble_test = []

for i in range(3):
    with K.tf.device('/gpu:0'):
        mdl = load_model(x_train)
        mdl.fit(x_train, y_train, nb_epoch=17, batch_size=4000,validation_split=0.1)
        ensemble_train.append(mdl.predict(x_train))
        ensemble_test.append(mdl.predict(x_test))    

/root/strawberry/code/keras_helper.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(540, activation="relu", kernel_initializer="glorot_uniform", input_shape=(9712,))`
  model.add(Dense(540, init='glorot_uniform', activation='relu', input_shape=(x_train.shape[1],)))
/root/strawberry/code/keras_helper.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(180, activation="relu", kernel_initializer="glorot_uniform")`
  model.add(Dense(180, init='glorot_uniform', activation='relu'))
/root/anaconda3/envs/ml_python/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 35796 samples, validate on 3978 samples
Epoch 1/17
35796/35796 [==============================] - 4s 124us/step - loss: 2.7307 - acc: 0.2044 - val_loss: 2.4217 - val_acc: 0.3115
Epoch 2/17
35796/35796 [==============================] - 3s 91us/step - loss: 2.3725 - acc: 0.3451 - val_loss: 2.0717 - val_acc: 0.4522
Epoch 3/17
35796/35796 [==============================] - 3s 93us/step - loss: 2.0758 - acc: 0.4405 - val_loss: 1.8238 - val_acc: 0.4947
Epoch 4/17
35796/35796 [==============================] - 3s 95us/step - loss: 1.8433 - acc: 0.4956 - val_loss: 1.6071 - val_acc: 0.5473
Epoch 5/17
35796/35796 [==============================] - 3s 95us/step - loss: 1.6324 - acc: 0.5438 - val_loss: 1.4000 - val_acc: 0.6031
Epoch 6/17
35796/35796 [==============================] - 3s 95us/step - loss: 1.4583 - acc: 0.5888 - val_loss: 1.2361 - val_acc: 0.6383
Epoch 7/17
35796/35796 [==============================] - 3s 94us/step - loss: 1.3121 - acc: 0.6266 - val_loss: 1.1083 - val_acc

In [12]:
result = lb.inverse_transform(sum(ensemble_test))
save_submission(testdf, result)

저장했다.


In [13]:
result = lb.inverse_transform(sum(ensemble_train))

In [14]:
traindf['predictions'] = result

In [15]:
from collections import Counter

In [16]:
total = Counter(traindf['cuisine'].tolist())
error = Counter(traindf['cuisine'][traindf['predictions'] != traindf['cuisine']].tolist())

In [17]:
traindf['cuisine'][traindf['predictions'] != traindf['cuisine']].tolist().count('korean')

138

In [18]:
for t,e in zip(total,error):
    if error[e]/total[t] > 0.2:
        print(str(t),"의 오답개수는",str(error[e]))
        print(str(t),"의 오답률은 ",str(error[e]/total[t]))             
        print("\n")

british 의 오답개수는 443
british 의 오답률은  0.5509950248756219


filipino 의 오답개수는 294
filipino 의 오답률은  0.3894039735099338


jamaican 의 오답개수는 107
jamaican 의 오답률은  0.20342205323193915


irish 의 오답개수는 229
irish 의 오답률은  0.34332833583208394


russian 의 오답개수는 233
russian 의 오답률은  0.47648261758691207


moroccan 의 오답개수는 263
moroccan 의 오답률은  0.32034104750304504


vietnamese 의 오답개수는 260
vietnamese 의 오답률은  0.3151515151515151


spanish 의 오답개수는 269
spanish 의 오답률은  0.27199191102123355


japanese 의 오답개수는 408
japanese 의 오답률은  0.286718200983837


